# Work with LinkedIn API in Python
---

---

<img src="images/logo.png" width="55%">

LinkedIn is the worlds largest business social networking hub. Launched in 2003, LinkedIn has millions of users and is implemented in over 200 countries. One purpose of the site is to allow registered users to maintain a list of contact details of people with whom they have some level of relationship, called Connections. Users can invite anyone (whether a site user or not) to become a connection. LinkedIn actually provides 2 APIs: 

- The **JavaScript API** is a rich client library enabling you to build dynamic applications in the web browser. Use OAuth 2 to easily authorize users via the "Sign In with LinkedIn" button, access LinkedIn data with native objects, and interact with Plugins.
- The **REST API** provides a simple, consistent representation of people, companies, jobs, and the interactions and relationships between them. Our query language lets you read data in XML and JSON at the granularity and aggregation that you choose. Use OAuth 1.0a to authorize users and begin making REST API calls using any programming language.

## Register a new application

If you do not already have an account with LinkedIn, go to https://www.linkedin.com and register. 

To create a LinkedIn App, go and visit: https://www.linkedin.com/secure/developer.

You need to be logged in to your LinkedIn account to be able to access their developer page. So first, login to your LinkedIn account and then follow the link above to get to their developer page. This should look similar to the one shown in the image below:

<img src="images/app_1.jpg">

On that page click on the "Create Application" link. Now the "Create Application" page does load. On that page enter all the information which do contain a star. They are required.

<img src="images/app_2.jpg">

Within the Application Info make sure, that you enter the correct URL of your website and that the URL you enter there does match 100% the one from your website. So if the URL of your website does include the www like `http://www.my_own_site.com` than you need to enter it like that. If the URL of your website doesn't include the www, than you don't need to include the www within the URL. It's important that the entered Website URL does match the one from your actual website, otherwise the App won't work correctly and it will show an error message when you or your users try to connect with the App.

After you entered all the required information, check the box at the bottom of the page to agree to the LinkedIn API Terms of Use and click on the "Submit" button.

After you added the application you will see a app listing page which will contain all the needed information to integrate that App in to your website. See image below:

<img src="images/app_3.jpg">

Use the "OAuth 2.0 Redirect URLs" field same as the LinkedIn callback url. Please note that:

* URLs must be absolute (e.g. "https://example.com/auth/callback", not "/auth/callback").
* URL arguments are ignored (i.e. https://example.com/?id=1 is the same as https://example.com/).
* URLs cannot include #'s (i.e. "https://example.com/auth/callback#linkedin" is invalid).

> Note, you may set many URLs for redirection. Let's set one of them such `http://localhost:8000`, because we will use it further. 

If these entries do not completely match the URL of your website, the application won't work and it will show an error message whe you try to connect to the App.

In [ ]:
# Credentials you get from registering a new application
client_id = "<YOUR_CLIENT_ID>"
client_secret = "<YOUR_CLIENT_SECRET>"

# Set an URL for redirection
redirect_url = "http://localhost:8000"

## Getting of access token

Once your application is properly configured, it's time to request an authorization code. The authorization code is used in the next step of the OAuth 2.0 flow to exchange for an actual access token. This is an important step because it provides assurance directly from LinkedIn to the user that permission is being granted to the correct application, with the agreed-upon access to the member's LinkedIn profile.

To request an authorization code, you must direct the user's browser to LinkedIn's OAuth 2.0 authorization endpoint (how it can be done with the help of Python is shown below):

    https://www.linkedin.com/uas/oauth2/authorization
    
Once the request is made, one of the following two situations will occur:

* If the user has not previously accepted the application's permission request, or the grant has expired or been manually revoked by the user, the browser will be redirected to LinkedIn's authorization screen (as seen in the picture below).  When the user completes the authorization process, the browser is redirected to the URL provided in the redirect_uri query parameter.

* If there is a valid existing permission grant from the user, the authorization screen is by-passed and the user is immediately redirected to the URL provided in the redirect_uri query parameter.

<img src="images/app_4.jpg">

By providing valid LinkedIn credentials and clicking on the "Allow Access" button, the user is approving your application's request to access their member data and interact with LinkedIn on their behalf.  This approval instructs LinkedIn to redirect the user back to the callback URL that you defined in your `redirect_uri` parameter. 

Attached to the redirect_uri will be two important URL arguments that you need to read from the request:

* `code` — The OAuth 2.0 authorization code (value that you will exchange with LinkedIn for an actual OAuth 2.0 access token in the next step of the authentcation process).
* `state` — A value used to test for possible CSRF attacks (before you accept the authorization code, your application should ensure that the value returned in the state parameter matches the state value from your original authorization code request).

The final step towards obtaining an Access Token is for your application to ask for one using the Authorization Code it just acquired. This is done by making the following "x-www-form-urlencoded" HTTP POST request:

    https://www.linkedin.com/uas/oauth2/accessToken
    
A successful Access Token request will return a JSON object containing the following fields:

* `access_token` — The access token for the user. This value must be kept secure, as per your agreement to the API Terms of Use.
* `expires_in` — The number of seconds remaining, from the time it was requested, before the token will expire.  

> Currently, all access tokens are issued with a 60 day lifespan.

Once you've obtained an Access Token, you can start making authenticated API requests on behalf of the user. This is accomplished by including an "Authorization" header in your HTTP call to LinkedIn's API. 

Let's make described above using Python library `requests` and `requests_oauthlib`. To install them (if they are absent) you can use pip:

    pip install requests requests_oauthlib

In [ ]:
# OAuth endpoints given in the LinkedIn API documentation
authorization_base_url = 'https://www.linkedin.com/uas/oauth2/authorization'
token_url = 'https://www.linkedin.com/uas/oauth2/accessToken'

import requests
from requests_oauthlib import OAuth2Session
from requests_oauthlib.compliance_fixes import linkedin_compliance_fix


LinkedIn = OAuth2Session(client_id, redirect_uri=redirect_url)
LinkedIn = linkedin_compliance_fix(LinkedIn)

# Redirect user to LinkedIn for authorization
authorization_url, state = LinkedIn.authorization_url(authorization_base_url)
print "State:", state
print 'Please go here and authorize,', authorization_url

# After clicking on the "Allow Access" button you will be redirected to the 
# http://localhost:8000/?code=... page
# So, to get the authorization verifier code from the callback url copy 
# URL from browser address line
redirect_response = raw_input('\nPaste the full redirect URL here:')

> You should note that Oauth2 works through SSL layer. If your server is not parametrized to allow HTTPS, the fetch_token method will raise an `oauthlib.oauth2.rfc6749.errors.InsecureTransportError`. You can disable this check by setting an environment variable:

In [ ]:
# Miss this step if the running of the next command cell works without errors
import os
os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

In [ ]:
# Fetch the access token
LinkedIn.fetch_token(token_url, client_secret=client_secret, authorization_response=redirect_response)
print LinkedIn.token, '\n'

# Let's remember access token
access_token = LinkedIn.token['access_token']

# Fetch a protected resource, i.e. user profile
r = LinkedIn.get('https://api.linkedin.com/v1/people/~?format=json')
print r.content

In the [similar way](http://requests-oauthlib.readthedocs.org/en/latest/examples/facebook.html) we could get user access token for Facebook in the respective lesson, but Facebook provides more simple tools to do this.

All over functional are coded in `python-linkedin` library, which provides a pure Python interface to the LinkedIn Profile, Group, Company, Jobs, Search, Share, Network and Invitation REST APIs. It's a best practice of using this (or any other) library for interaction with LinkedIn API. 

LinkedIn provides a service that lets people bring their LinkedIn profiles and networks with them to your site or application via their OAuth based API. This library provides a lightweight interface over a complicated LinkedIn OAuth based API to make it for python programmers easy to use.

You can install python-linkedin library via pip:

    pip install python-linkedin
    
Here and further we will consider and use only `python-linkedin` library. 

In [ ]:
from linkedin import linkedin

# Starting from May 12, 2015, Linkedin has limited the open APIs. 
# See here: https://developer.linkedin.com/support/developer-program-transition
# Particularly. access to r_fullprofile scopes requires that you apply for 
# and are granted access to this information from LinkedIn. 
# The last updated version 4.1 of *python-linkedin* library doesn't take into account these changes.
# Thus, we will make it ourselves
# linkedin.PERMISSIONS.enums should be 
# {
#  'BASIC_PROFILE': 'r_basicprofile', 
#  'NETWORK_UPDATES': 'w_share'
# }

for key in linkedin.PERMISSIONS.enums.keys():
    if key == 'BASIC_PROFILE':
        linkedin.PERMISSIONS.enums[key] = 'r_basicprofile'
    elif key == 'NETWORK_UPDATES':
        linkedin.PERMISSIONS.enums[key] = 'w_share'
    else:
        del linkedin.PERMISSIONS.enums[key]

authentication = linkedin.LinkedInAuthentication(client_id, client_secret, redirect_url, linkedin.PERMISSIONS.enums.values())
print 'Please go here and authorize,', authentication.authorization_url

redirect_response = raw_input('\nPaste the full redirect URL here:')
authorization_code = redirect_response.split("code=")[1].split("&")[0]
print "\nAuthorization Code:", authorization_code

In [ ]:
# After setting of the authorization code by hand, we can call the *get_access_token()* to get the actual token.
authentication.authorization_code = authorization_code
AT = authentication.get_access_token()
AT

In [ ]:
access_token = AT.access_token

After you get the access token, you are now permitted to make API calls on behalf of the user who granted access to you app. In addition to that, in order to prevent from going through the OAuth flow for every consecutive request, one can directly assign the access token obtained before to the application instance.

In [ ]:
app = linkedin.LinkedInApplication(token=access_token)
app

> For testing the library using an interpreter, use the quick helper.
>
>`    from linkedin import server`<br></br>
>`    app = server.quick_api(KEY, SECRET)`
>
> This will print the authorization url to the screen. Go into this URL using a browser, after you login, the method will return with an API object you can now use.
>
>`    app.get_profile()`

In [ ]:
# Let's use the app and get your profile
app.get_profile()

In [ ]:
# You may get LinkedIn profile of any other user
# For example, Bill Gates profile by url
bill_gates_url = 'https://www.linkedin.com/in/williamhgates'
app.get_profile(member_url=bill_gates_url)

In [ ]:
#Bill Gates profile by id
bill_gates_id = 'UN_vks5Pmj'
app.get_profile(member_id=bill_gates_id)

In [ ]:
# Get and display avatar of any user
avatar = app.get_picture_urls(member_id=bill_gates_id)
avatar

In [ ]:
from IPython.core.display import HTML

HTML('<p><img src="{}" width="300px"></p>'.format(avatar["values"][0]))

In [ ]:
# Let's get LinkedIn info about Microsoft (its id is 1035)
app.get_companies(company_ids=[1035])